# Analisi - Grafi

In [1]:
import pickle
from shapely import Point
import geopandas as gpd

from my_paths import *
import Graph_functions

## Analisi Automatica - Documentazione

Tramite la funzione **Graph_functions.auto_analysis_poi()** è possibile eseguire in automatico la ricreazione del grafo stradale con il dizionario dei pesi desiderato. Spiegazione della funzione:
 ```python
(function) def auto_analysis_poi(
    list_gdfs_poi: list[dict],
    custom_weights: dict = None,
    PATH_GEOJSON: str = None,
    PATH_PICKLE: str = None
) -> None
```
- Richiede una struttura a liste di dizionari di geoDataFrame per "poi" del tipo:
    ```python
    gdf_list = [
        {"gdf":  gpd.read_file(PATH_DEL_GEOJSON),
        "tipo": "tipo che si vuole assegnare agli archi che connettono il gdf al grafo",
        "attr": {"nome_attributo": "valore_attributo"} # Opzionale
        }
    ]
    ```
    Un attributo che però **non è opzionale** **se** si vuole eseguire la nuova funzione di **"auto_filter_poi"** è **"priorita"** che deve essere un numero  
    intero da 1 in poi. Minore è il valore e maggiore è la priorità (priorità 1 = massima).
    ```python
    "attr": {"priorita": 1}
- custom_weights: Un dizionario di pesi del tipo:
    ```python
    {"cycleway": 0, "primary": 1, ....}
    ```
    Possibilità di inserire come chiave: "default": quando non viene trovata una "highway" associta al dizionario,  
    verrà utilizzato il valore contenuto in "default".  
    In questo modo se si fornisce un dizionario in cui c'è soltanto la chiave "default", tutte le strade avranno quel  
    valore associato (esempio per creare un grafo di strade senza pesi):
    ```python
    {"default": 1}
    ```
    Se non viene dato nessun dizionario di pesi, verrà utilizzato direttamente il grafo salvato in staging, che contiene i pesi
    standard che abbiamo stabilito, salvati su file json.
- Si possono inserire i vari percorsi di salvataggio dei file pickle (grafo) e geojson(geoDataFrame).
    

    ```

Documentazione per **"auto_filter_poi"**
Tramite un grafo (il pickle) generato da un **Graph_functions.auto_analysis_poi()** è possibile lanciare una versione "ridotta" dei percorsi
in base a un numero fissato di "poi".  
Attributi:
```python
(function) def auto_filter_poi(
    G_percorsi: MultiDiGraph,
    max_pois: int,
    peso_inquinamento_vs_incidenti: float = 0.5,
    peso_fattori_esterni_vs_distribuzione_poi: float = 0.5,
    custom_weights: dict = None,
    PATH_GEOJSON: str = None,
    PATH_PICKLE: str = None
) -> None
```
- **G_percorsi** -> Il file pickle salvato da **Graph_functions.auto_analysis_poi()**
- **max_pois** -> Il numero massimo (circa) di "pois" su cui creare la versione ridotta del grafo originale.
- **peso_inquinamento_vs_incidenti** -> Un numero da 0 -> 1: Più basso è e più sarà orientato verso **incidenti** (priorità su inquinamento = 0 per intenderci), mentre al contrario più è alto e più sarà sbilanciato su **inquinamento** rispetto a incidenti. Di base + 0.5 (non sbilanciato).
- **peso_fattori_esterni_vs_distribuzione_poi** -> Stesso discorso di prima ma per quanto riguarda l'impatto dei fattori visti prima, denominati nel loro insieme come **fattori_esterni**. Per **distribuzione_poi** intendiamo la loro "proporzionale" diminuzione in base alla loro normale distribuzione tra i vari municipi.  
Esempio: Municipio 1 ha 10 pois e Municipio 2 ne ha 20 (tot 30) e noi vogliamo un max di 10 -> vista la proporzione di 1 a 2 ->  
Municipio 1 ottiene circa 3 poi mentro Municipio 2 ne ottiene circa 7. Questo nel caso in cui si mettesse come valore 1 a questo attributo e quindi nel caso in cui i "fattori esterni" non vengano tenuti in considerazione, altrimenti potrebbero andare a cambiare questi numeri in base a quanto peso si è deciso di dargli nel computo e poi di conseguenza a quale dei due fattori si è deciso di dare più importanza tramite il precedente parametro.
- Gli altri parametri sono uguali a **auto_filter_poi()**

## Estensione rete ciclabili

ETL gdf:  
Provo a creare dei percorsi che hanno lo scopo di unire tutti i frammenti di ciclabili esistenti.  
Quindi userò come "poi" tutti i nodi delle "highway"=cycleway

In [2]:
gdf_ciclabili = gpd.read_file(PATH_STRADE_CICLABILI_GEOJSON_CLEAN)
gdf_ciclabili_cycleway = gdf_ciclabili[gdf_ciclabili["highway"] == "cycleway"]
gdf_point_ciclabili = []
for idx, row in gdf_ciclabili_cycleway.iterrows():
    gdf_point_ciclabili.append(Point(row.geometry.coords[0]))
    gdf_point_ciclabili.append(Point(row.geometry.coords[-1]))
gdf_point_ciclabili = gpd.GeoDataFrame(gdf_point_ciclabili, columns=["geometry"], crs=CRS_GRAD)

### Analisi Completo

In [3]:
gdf_list = [
    {"gdf":  gdf_point_ciclabili,
     "tipo": "ciclabile",
     "attr": {"priorita": 1}}
]
Graph_functions.auto_analysis_poi(gdf_list,
                                  PATH_PICKLE=PATH_EXTENDED_CICLABILI_PICKLE_STAGING,
                                  PATH_GEOJSON=PATH_EXTENDED_CICLABILI_CLEAN)

# --------------------  Post ETL  --------------------------#
# Rimuoviamo gli archi artificiali (in questo caso non ci sono effetti poi che abbiamo collegato)
gdf_extended_ciclabili = gpd.read_file(PATH_EXTENDED_CICLABILI_CLEAN)
gdf_extended_ciclabili = gdf_extended_ciclabili[gdf_extended_ciclabili["artificial"]==False]
gdf_extended_ciclabili.reset_index(drop=True).to_file(PATH_EXTENDED_CICLABILI_CLEAN, driver="GeoJSON")

### Analisi con filtri

Utilizzare questo per capire quanti "poi" contiene il gdf e scegliere i "max pois" da inserire

In [4]:
len(gdf_point_ciclabili)

9686

In [66]:
with open(PATH_EXTENDED_CICLABILI_PICKLE_STAGING, "rb") as file:
    G_extended_ciclabili = pickle.load(file)

n_pois = len(gdf_point_ciclabili) / 100 * 30    # 30% del totale
Graph_functions.auto_filter_poi(G_percorsi = G_extended_ciclabili,
                                max_pois= n_pois,
                                peso_inquinamento_vs_incidenti = 0,
                                peso_fattori_esterni_vs_distribuzione_poi = 1,
                                PATH_GEOJSON = PATH_EXTENDED_CICLABILI_FILTRATO_CLEAN)

# --------------------  Post ETL  --------------------------#
# Rimuoviamo gli archi artificiali (in questo caso non ci sono effetti poi che abbiamo collegato)
gdf_extended_ciclabili = gpd.read_file(PATH_EXTENDED_CICLABILI_FILTRATO_CLEAN)
gdf_extended_ciclabili = gdf_extended_ciclabili[gdf_extended_ciclabili["artificial"]==False]
gdf_extended_ciclabili.reset_index(drop=True).to_file(PATH_EXTENDED_CICLABILI_FILTRATO_CLEAN, driver="GeoJSON")

### Statistiche

Per praticità per ora basta commentare quello di cui non si vuole guardare le statistiche

In [74]:
gdf_extended_ciclabili = gpd.read_file(PATH_EXTENDED_CICLABILI_CLEAN)
#gdf_extended_ciclabili = gpd.read_file(PATH_EXTENDED_CICLABILI_FILTRATO_CLEAN)

1. Km per ogni "highway"

In [75]:
gdf_extended_ciclabili_highway = gdf_extended_ciclabili.to_crs(CRS_METR).groupby("highway").agg({
    "length": lambda x: sum(x/1000)
    }).sort_values(by="length", ascending=False)
print(gdf_extended_ciclabili_highway.head(6))
print(gdf_extended_ciclabili_highway["length"].sum(), "km")

                 length
highway                
cycleway     206.214018
footway       37.048211
residential   15.299822
secondary     14.007885
tertiary      11.685893
service        5.018637
306.56043468606293 km


2. Km non di "ciclabile", quindi da costruire

In [76]:
float(gdf_extended_ciclabili[gdf_extended_ciclabili["ciclabile"] == "False"]["length"].sum()/1000)

94.60126123978074

## Sport e tempo libero

Utilizzeremo i seguenti dati:
1. **Parchi**
2. **Impianti sportivi**
3. **Aree gioco**  

Saranno i nostri "poi", cioè "Point of Interest" che verranno aggiunti al Grafo della rete Ciclabile/Stradale per la ricerca di percorsi

### Analisi completa

In [5]:
gdf_list = [
    {"gdf":  gpd.read_file(PATH_PARCHI_CLEAN),
     "tipo": "parchi",
     "attr": {"priorita": 1}},
     {"gdf":  gpd.read_file(PATH_IMPIANTI_SPORTIVI_CLEAN),
     "tipo": "impianti_sportivi",
     "attr": {"priorita": 2}},
     {"gdf":  gpd.read_file(PATH_AREE_GIOCO_CLEAN),
     "tipo": "aree_gioco",
     "attr": {"priorita": 3}} 
]
Graph_functions.auto_analysis_poi(gdf_list,
                                  PATH_PICKLE=PATH_SPORT_TEMPO_LIBERO_PICKLE_STAGING,
                                  PATH_GEOJSON=PATH_SPORT_TEMPO_LIBERO_CLEAN)

### Analisi filtrata

Incidenti

In [6]:
with open(PATH_SPORT_TEMPO_LIBERO_PICKLE_STAGING, "rb") as file:
    G_sport = pickle.load(file)

Graph_functions.auto_filter_poi(G_percorsi=G_sport,
                                max_pois=400,
                                peso_inquinamento_vs_incidenti=0,
                                peso_fattori_esterni_vs_distribuzione_poi=1,
                                PATH_GEOJSON=PATH_SPORT_TEMPO_LIBERO_FILTRATO_CLEAN.replace("clean.geojson", "incidenti_clean.geojson"))

Inquinamento

In [13]:
with open(PATH_SPORT_TEMPO_LIBERO_PICKLE_STAGING, "rb") as file:
    G_sport = pickle.load(file)

Graph_functions.auto_filter_poi(G_percorsi=G_sport,
                                max_pois=400,
                                peso_inquinamento_vs_incidenti=1,
                                peso_fattori_esterni_vs_distribuzione_poi=1,
                                PATH_GEOJSON=PATH_SPORT_TEMPO_LIBERO_FILTRATO_CLEAN.replace("clean.geojson", "inquinamento_clean.geojson"))

Unione dei percorsi in un unico file. Aggiungo dei Tag per distinguere i dataframe.

In [21]:
gdf_completo = gpd.read_file(PATH_SPORT_TEMPO_LIBERO_CLEAN)
gdf_filtrato_incidenti = gpd.read_file(PATH_SPORT_TEMPO_LIBERO_FILTRATO_CLEAN.replace("clean.geojson", "incidenti_clean.geojson"))
gdf_filtrato_inquinamento = gpd.read_file(PATH_SPORT_TEMPO_LIBERO_FILTRATO_CLEAN.replace("clean.geojson", "inquinamento_clean.geojson"))

gdf_completo["origine"] = "completo"
gdf_filtrato_incidenti["origine"] = "solo_incidenti"
gdf_filtrato_inquinamento["origine"] = "solo_inquinamento"
gdf_unito = gpd.GeoDataFrame(gpd.pd.concat([
        gdf_completo,
        gdf_filtrato_incidenti,
        gdf_filtrato_inquinamento,
    ], ignore_index=True), crs=CRS_GRAD)

In [19]:
gdf_unito.to_file(PATH_SPORT_TEMPO_LIBERO_CLEAN.replace("clean.geojson", "unito_clean.geojson"))

### Statisiche

In [104]:
gdf_analisi_sport = gpd.read_file(PATH_SPORT_TEMPO_LIBERO_CLEAN)

In [109]:
gdf_analisi_sport_mod = gdf_analisi_sport[gdf_analisi_sport["artificial"] != True]
gdf_analisi_sport_mod[gdf_analisi_sport_mod["ciclabile"]=="True"]["length"].sum()/1000

np.float64(104.9339840532247)

pre pulizia per tutte le statistiche

In [108]:
# Eliminiamo connessioni "artificiali" usate per connettere i "poi" al grafo delle strade
gdf_analisi_sport_mod = gdf_analisi_sport[gdf_analisi_sport["artificial"] != True]

# Eliminiamo i percorsi completamente contenuti all'interno dei "poi" poligonari: parchi
gdf = gpd.read_file(PATH_PARCHI_CLEAN)
geometry_parchi = gdf.union_all()
gdf_senza_parchi = []
for idx, row in gdf_analisi_sport_mod.iterrows():
    if not geometry_parchi.contains(row.geometry):
        gdf_senza_parchi.append(row)
gdf_analisi_sport_mod = gpd.GeoDataFrame(gdf_senza_parchi, crs=CRS_GRAD)

1. Km per "highway"

In [102]:
# Raggruppiamo per "highway" e calcoliamo (in km -> /1000) la lunghezza complessiva delle strade
gdf_analisi_sport_mod_most_highways = gdf_analisi_sport_mod.to_crs(CRS_METR).groupby("highway").agg({
    "length": lambda x: sum(x/1000)
    }).sort_values(by="length", ascending=False)
# Stampiamo il risultato
print(gdf_analisi_sport_mod_most_highways)
print(f"Tot: {gdf_analisi_sport_mod_most_highways["length"].sum()}")

                                 length
highway                                
footway                      179.947934
cycleway                     102.764815
residential                   38.399767
tertiary                      24.759123
secondary                     10.237838
primary                        7.072719
service                        5.422086
pedestrian                     4.235049
path                           3.516888
track                          2.303546
footway service                1.271002
unclassified                   1.251304
footway steps                  0.741803
footway path                   0.569967
residential footway            0.474065
footway pedestrian             0.466265
proposed footway service       0.321046
cycleway path                  0.291043
residential pedestrian         0.275658
cycleway footway               0.272785
track service                  0.234884
tertiary_link                  0.217631
residential cycleway           0.214467


2. Km non di "ciclabile", quindi da costruire

In [95]:
float(gdf_analisi_sport["length"].sum()/1000)

450.50374066181047

In [88]:
float(gdf_analisi_sport_mod[gdf_analisi_sport_mod["ciclabile"] == "False"]["length"].sum()/1000)

112.65026341818994

Numero poi per municipio che sono stati connessi e % poi connessi in ogni municipio rispetto al totale connesso

In [89]:
gdf_analisi_sport_raggruppato = gdf_analisi_sport.groupby(["MUNICIPIO"]).agg({
    "poi": "sum",
    "length": lambda x: sum(x)/1000
}).reset_index()
gdf_analisi_sport_raggruppato["MUNICIPIO"] = gdf_analisi_sport_raggruppato["MUNICIPIO"].astype(int)
gdf_analisi_sport_raggruppato = gdf_analisi_sport_raggruppato.sort_index()
gdf_analisi_sport_raggruppato = gdf_analisi_sport_raggruppato.merge(
    gpd.read_file(PATH_INCIDENTI_CLEAN)[["MUNICIPIO", "Incidenti"]], on="MUNICIPIO")

gdf_pois = gpd.read_file(PATH_SCORE_X_MUNICIPI)[["MUNICIPIO", "parchi", "impianti_sportivi", "aree_gioco"]]
gdf_pois["tot"] = gdf_pois.drop("MUNICIPIO", axis=1).sum(axis=1)
gdf_pois["MUNICIPIO"] = gdf_pois["MUNICIPIO"].astype(int)
gdf_analisi_sport_raggruppato = gdf_analisi_sport_raggruppato.merge(gdf_pois[["MUNICIPIO", "tot"]], on="MUNICIPIO")
somma_poi = gdf_analisi_sport_raggruppato["poi"].sum()
gdf_analisi_sport_raggruppato["% poi"] = gdf_analisi_sport_raggruppato.apply(lambda row: row["poi"] / somma_poi * 100, axis=1)
gdf_analisi_sport_raggruppato.sort_values(["poi", "Incidenti", "length"], ascending=False, inplace=True)
print(gdf_analisi_sport_raggruppato)


   MUNICIPIO  poi     length  Incidenti  tot  % poi
2          3   56  28.429212       1014   74  14.00
7          8   55  30.266064        996  155  13.75
8          9   52  28.539204        947  185  13.00
0          1   48  23.733266        859   49  12.00
6          7   46  29.452303        806  127  11.50
5          6   39  20.985732        698  127   9.75
3          4   38  25.111688        691  112   9.50
4          5   35  27.389635        638  104   8.75
1          2   31  21.270263        567   84   7.75


Dettaglio sulla quantità di "tipo di poi"

In [ ]:
gdf_analisi_sport = gdf_analisi_sport[gdf_analisi_sport["tipo"] != "Strade_ciclabili"]
gdf_analisi_sport_raggruppato = gdf_analisi_sport.groupby("tipo").agg({
    "poi": "sum"}).reset_index().sort_values(["poi"], ascending=[False])
gdf_analisi_sport_raggruppato

## Istruzione

Utilizzeremo i seguenti dati:
1. **Biblioteche**
2. **Scuole**
Saranno i nostri "poi", cioè "Point of Interest" che verranno aggiunti al Grafo della rete Ciclabile/Stradale per la ricerca di percorsi

### Analisi Completa

In [7]:
gdf_list = [
    {"gdf":  gpd.read_file(PATH_BIBLIOTECHE_CLEAN),
     "tipo": "Biblioteche",
     "attr": {"priorita": 1}},
     {"gdf":  gpd.read_file(PATH_SCUOLE_CLEAN),
     "tipo": "Scuole",
     "attr": {"priorita": 2}}
     ]
Graph_functions.auto_analysis_poi(gdf_list, PATH_GEOJSON=PATH_ISTRUZIONE_CLEAN)

### Analisi Filtrata

In [9]:
with open(PATH_ISTRUZIONE_PICKLE_STAGING, "rb") as file:
    G_istruzione = pickle.load(file)

Graph_functions.auto_filter_poi(G_percorsi=G_istruzione,
                                max_pois=400,
                                peso_inquinamento_vs_incidenti=0,
                                peso_fattori_esterni_vs_distribuzione_poi=1,
                                PATH_GEOJSON=PATH_ISTRUZIONE_FILTRATO_CLEAN.replace("clean.geojson", "incidenti_clean.geojson"))

In [10]:
with open(PATH_ISTRUZIONE_PICKLE_STAGING, "rb") as file:
    G_istruzione = pickle.load(file)

Graph_functions.auto_filter_poi(G_percorsi=G_istruzione,
                                max_pois=400,
                                peso_inquinamento_vs_incidenti=1,
                                peso_fattori_esterni_vs_distribuzione_poi=1,
                                PATH_GEOJSON=PATH_ISTRUZIONE_FILTRATO_CLEAN.replace("clean.geojson", "inquinamento_clean.geojson"))

In [17]:
gdf_completo = gpd.read_file(PATH_ISTRUZIONE_CLEAN)
gdf_filtrato_incidenti = gpd.read_file(PATH_ISTRUZIONE_FILTRATO_CLEAN.replace("clean.geojson", "incidenti_clean.geojson"))
gdf_filtrato_inquinamento = gpd.read_file(PATH_ISTRUZIONE_FILTRATO_CLEAN.replace("clean.geojson", "inquinamento_clean.geojson"))

gdf_completo["origine"] = "completo"
gdf_filtrato_incidenti["origine"] = "solo_incidenti"
gdf_filtrato_inquinamento["origine"] = "solo_inquinamento"
gdf_unito = gpd.GeoDataFrame(gpd.pd.concat([
        gdf_completo,
        gdf_filtrato_incidenti,
        gdf_filtrato_inquinamento,
    ], ignore_index=True), crs=CRS_GRAD)

In [18]:
gdf_unito.to_file(PATH_ISTRUZIONE_FILTRATO_CLEAN.replace("clean.geojson", "unito_clean.geojson"))

### Statisiche

In [22]:
gdf_analisi_istruzione = gpd.read_file(PATH_ISTRUZIONE_CLEAN, driver= "GeoJSON")

c:\Users\apera\OneDrive\Documenti\progetti\Progetto-Bike\.venv\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver GeoJSON does not support open option DRIVER
  return ogr_read(


In [23]:
# Eliminiamo connessioni "artificiali" usate per connettere i "poi" al grafo delle strade
gdf_analisi_istruzione_mod = gdf_analisi_istruzione[gdf_analisi_istruzione["artificial"] != True]

# Leggi i due POI
gdf_scuole = gpd.read_file(PATH_SCUOLE_CLEAN)
gdf_biblioteche = gpd.read_file(PATH_BIBLIOTECHE_CLEAN)

# Unisci in un'unica geometria
geometry_poi = gpd.GeoSeries(
    gpd.pd.concat([gdf_scuole.geometry, gdf_biblioteche.geometry], ignore_index=True)
).union_all()

# Filtra il GDF di partenza
gdf_senza_poi = []
for idx, row in gdf_analisi_istruzione_mod.iterrows():
    if not geometry_poi.contains(row.geometry):
        gdf_senza_poi.append(row)

# Crea nuovo GeoDataFrame
gdf_analisi_istruzione_mod = gpd.GeoDataFrame(gdf_senza_poi, crs=CRS_GRAD)

# Raggruppiamo per "highway" e calcoliamo (in km -> /1000) la lunghezza complessiva delle strade
gdf_analisi_istruzione_mod_most_highways = gdf_analisi_istruzione_mod.to_crs(CRS_METR).groupby("highway").agg({
    "length": lambda x: sum(x/1000)
    }).sort_values(by="length", ascending=False)

In [24]:
# Stampiamo il risultato
print(gdf_analisi_istruzione_mod_most_highways)
print(f"Tot: {gdf_analisi_istruzione_mod_most_highways["length"].sum()}")

                              length
highway                             
footway                   109.403574
cycleway                   85.216865
residential                33.887987
tertiary                   23.981052
secondary                  12.057483
primary                     6.631249
track                       3.158726
pedestrian                  2.210088
service                     1.978385
path                        1.692647
footway steps               0.476395
unclassified                0.306090
residential footway         0.233284
secondary_link              0.224865
track service path          0.217083
footway service             0.207583
tertiary_link               0.175564
residential cycleway        0.169111
footway pedestrian          0.158530
residential pedestrian      0.146128
cycleway path               0.133034
cycleway footway            0.112219
cycleway service            0.093463
primary_link                0.085114
living_street               0.078441
r

In [ ]:
float(gdf_analisi_istruzione_mod_most_highways[gdf_analisi_istruzione_mod_most_highways["ciclabile"] == "False"]["length"].sum()/1000)

Numero poi per municipio che sono stati connessi e % poi connessi in ogni municipio rispetto al totale connesso

In [27]:
gdf_analisi_istruzione_raggruppato = gdf_analisi_istruzione.groupby(["MUNICIPIO"]).agg({
    "poi": "sum",
    "length": lambda x: sum(x)/1000
}).reset_index()
gdf_analisi_istruzione_raggruppato["MUNICIPIO"] = gdf_analisi_istruzione_raggruppato["MUNICIPIO"].astype(int)
gdf_analisi_istruzione_raggruppato = gdf_analisi_istruzione_raggruppato.sort_index()
gdf_analisi_istruzione_raggruppato = gdf_analisi_istruzione_raggruppato.merge(
    gpd.read_file(PATH_INCIDENTI_CLEAN)[["MUNICIPIO", "Incidenti"]], on="MUNICIPIO")

gdf_pois = gpd.read_file(PATH_SCORE_X_MUNICIPI)[["MUNICIPIO", "biblioteche", "scuole"]]
gdf_pois["tot"] = gdf_pois.drop("MUNICIPIO", axis=1).sum(axis=1)
gdf_pois["MUNICIPIO"] = gdf_pois["MUNICIPIO"].astype(int)
gdf_analisi_istruzione_raggruppato = gdf_analisi_istruzione_raggruppato.merge(gdf_pois[["MUNICIPIO", "tot"]], on="MUNICIPIO")
somma_poi = gdf_analisi_istruzione_raggruppato["poi"].sum()
gdf_analisi_istruzione_raggruppato["% poi"] = gdf_analisi_istruzione_raggruppato.apply(lambda row: row["poi"] / somma_poi * 100, axis=1)
gdf_analisi_istruzione_raggruppato.sort_values(["poi", "Incidenti", "length"], ascending=False, inplace=True)
print(gdf_analisi_istruzione_raggruppato)

   MUNICIPIO  poi     length  Incidenti  tot      % poi
0          1  373  35.595537        859  373  29.209084
2          3  252  35.426513       1014  252  19.733751
8          9  194  43.906813        947  194  15.191856
5          6   93  31.762571        698   93   7.282694
3          4   86  31.708193        691   86   6.734534
4          5   77  23.511159        638   77   6.029757
7          8   73  36.938777        996   73   5.716523
1          2   66  31.334217        567   66   5.168363
6          7   63  37.266871        806   63   4.933438


In [28]:
gdf_analisi_istruzione = gdf_analisi_istruzione[gdf_analisi_istruzione["tipo"] != "Strade_ciclabili"]
gdf_analisi_istruzione_raggruppato = gdf_analisi_istruzione.groupby("tipo").agg({
    "poi": "sum"}).reset_index().sort_values(["poi"], ascending=[False])
gdf_analisi_istruzione_raggruppato

,tipo,poi
1,Scuole,1251
0,Biblioteche,26


## Cultura e spettacolo

Utilizzeremo i seguenti dati: 

1. **Musei**
2. **Cinema**
3. **Teatri**

Saranno i nostri "poi", cioè "Point of Interest"
che verranno aggiunti al Grafo della rete 
Ciclabile/Stradale per la ricerca di percorsi

### Analisi Completa

In [21]:
gdf_list = [
    {"gdf":  gpd.read_file(PATH_MUSEI_CLEAN),
     "tipo": "musei",
     "attr": {"priorita": 1}},
     {"gdf":  gpd.read_file(PATH_CINEMA_CLEAN),
     "tipo": "cinema",
     "attr": {"priorita": 2}}, 
     {"gdf":  gpd.read_file(PATH_TEATRI_CLEAN),
     "tipo": "teatri",
     "attr": {"priorita": 3}}
     ]
Graph_functions.auto_analysis_poi(gdf_list,
                                  PATH_PICKLE= PATH_CULTURA_SPETTACOLO_PICKLE_STAGING,
                                  PATH_GEOJSON=PATH_CULTURA_SPETTACOLO_CLEAN)

### Analisi Filtrata

In [22]:
with open(PATH_CULTURA_SPETTACOLO_PICKLE_STAGING, "rb") as file:
    G_cultura_spettacolo = pickle.load(file)

Graph_functions.auto_filter_poi(G_percorsi=G_cultura_spettacolo,
                                max_pois=100,
                                peso_inquinamento_vs_incidenti=0,
                                peso_fattori_esterni_vs_distribuzione_poi=1,
                                PATH_GEOJSON=PATH_CULTURA_SPETTACOLO_FILTRATO_CLEAN.replace("clean.geojson", "incidenti_clean.geojson"))

In [23]:
with open(PATH_CULTURA_SPETTACOLO_PICKLE_STAGING, "rb") as file:
    G_cultura_spettacolo = pickle.load(file)

Graph_functions.auto_filter_poi(G_percorsi=G_cultura_spettacolo,
                                max_pois=100,
                                peso_inquinamento_vs_incidenti=1,
                                peso_fattori_esterni_vs_distribuzione_poi=1,
                                PATH_GEOJSON=PATH_CULTURA_SPETTACOLO_FILTRATO_CLEAN.replace("clean.geojson", "inquinamento_clean.geojson"))

In [24]:
gdf_completo = gpd.read_file(PATH_CULTURA_SPETTACOLO_CLEAN)
gdf_filtrato_incidenti = gpd.read_file(PATH_CULTURA_SPETTACOLO_FILTRATO_CLEAN.replace("clean.geojson", "incidenti_clean.geojson"))
gdf_filtrato_inquinamento = gpd.read_file(PATH_CULTURA_SPETTACOLO_FILTRATO_CLEAN.replace("clean.geojson", "inquinamento_clean.geojson"))

gdf_completo["origine"] = "completo"
gdf_filtrato_incidenti["origine"] = "solo_incidenti"
gdf_filtrato_inquinamento["origine"] = "solo_inquinamento"
gdf_unito = gpd.GeoDataFrame(gpd.pd.concat([
        gdf_completo,
        gdf_filtrato_incidenti,
        gdf_filtrato_inquinamento,
    ], ignore_index=True), crs=CRS_GRAD)

In [25]:
gdf_unito.to_file(PATH_CULTURA_SPETTACOLO_FILTRATO_CLEAN.replace("clean.geojson", "unito_clean.geojson"))

### Statistiche

In [8]:
gdf_analisi_cultura_spettacolo = gpd.read_file(PATH_CULTURA_SPETTACOLO_CLEAN, driver= "GeoJSON")

c:\Users\apera\OneDrive\Documenti\progetti\Progetto-Bike\.venv\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver GeoJSON does not support open option DRIVER
  return ogr_read(


In [9]:
# Eliminiamo connessioni "artificiali" usate per connettere i "poi" al grafo delle strade
gdf_analisi_cultura_spettacolo_mod = gdf_analisi_cultura_spettacolo[gdf_analisi_cultura_spettacolo["artificial"] != True]

# Leggi i due POI
gdf_musei = gpd.read_file(PATH_MUSEI_CLEAN)
gdf_teatri = gpd.read_file(PATH_TEATRI_CLEAN)
gdf_cinema = gpd.read_file(PATH_CINEMA_CLEAN)

# Unisci in un'unica geometria
geometry_poi = gpd.GeoSeries(
    gpd.pd.concat([gdf_cinema.geometry, gdf_teatri.geometry, gdf_musei.geometry], ignore_index=True)
).union_all()

# Filtra il GDF di partenza
gdf_senza_poi = []
for idx, row in gdf_analisi_cultura_spettacolo.iterrows():
    if not geometry_poi.contains(row.geometry):
        gdf_senza_poi.append(row)

# Crea nuovo GeoDataFrame
gdf_analisi_cultura_spettacolo_mod = gpd.GeoDataFrame(gdf_senza_poi, crs=CRS_GRAD)

# Raggruppiamo per "highway" e calcoliamo (in km -> /1000) la lunghezza complessiva delle strade
gdf_analisi_cultura_spettacolo_mod_most_highways = gdf_analisi_cultura_spettacolo_mod.to_crs(CRS_METR).groupby("highway").agg({
    "length": lambda x: sum(x/1000)
    }).sort_values(by="length", ascending=False)

In [10]:
# Stampiamo il risultato
print(gdf_analisi_cultura_spettacolo_mod_most_highways)
print(f"Tot: {gdf_analisi_cultura_spettacolo_mod_most_highways["length"].sum()}")

                         length
highway                        
cycleway              43.821006
footway               28.797061
residential           11.283660
secondary              7.617717
tertiary               5.698120
pedestrian             2.073513
primary                1.240658
footway steps          0.457708
service                0.443294
cycleway path          0.424077
path                   0.276330
unclassified           0.205090
secondary_link         0.154438
residential cycleway   0.116950
footway service        0.079077
primary_link           0.066293
cycleway footway       0.058737
footway pedestrian     0.048837
steps path             0.018337
Tot: 102.88090211247821


Numero poi per municipio che sono stati connessi e % poi connessi in ogni municipio rispetto al totale connesso

In [16]:
gdf_analisi_cultura_spettacolo_raggruppato = gdf_analisi_cultura_spettacolo.groupby(["MUNICIPIO"]).agg({
    "poi": "sum",
    "length": lambda x: sum(x)/1000
}).reset_index()
gdf_analisi_cultura_spettacolo_raggruppato["MUNICIPIO"] = gdf_analisi_cultura_spettacolo_raggruppato["MUNICIPIO"].astype(int)
gdf_analisi_cultura_spettacolo_raggruppato = gdf_analisi_cultura_spettacolo_raggruppato.sort_index()
gdf_analisi_cultura_spettacolo_raggruppato = gdf_analisi_cultura_spettacolo_raggruppato.merge(
    gpd.read_file(PATH_INCIDENTI_CLEAN)[["MUNICIPIO", "Incidenti"]], on="MUNICIPIO")

gdf_pois = gpd.read_file(PATH_SCORE_X_MUNICIPI)[["MUNICIPIO", "cinema", "musei", "teatri"]]
gdf_pois["tot"] = gdf_pois.drop("MUNICIPIO", axis=1).sum(axis=1)
gdf_pois["MUNICIPIO"] = gdf_pois["MUNICIPIO"].astype(int)
gdf_analisi_cultura_spettacolo_raggruppato = gdf_analisi_cultura_spettacolo_raggruppato.merge(gdf_pois[["MUNICIPIO", "tot"]], on="MUNICIPIO")
somma_poi = gdf_analisi_cultura_spettacolo_raggruppato["poi"].sum()
gdf_analisi_cultura_spettacolo_raggruppato["% poi"] = gdf_analisi_cultura_spettacolo_raggruppato.apply(lambda row: row["poi"] / somma_poi * 100, axis=1)
gdf_analisi_cultura_spettacolo_raggruppato.sort_values(["poi", "Incidenti", "length"], ascending=False, inplace=True)
print(gdf_analisi_cultura_spettacolo_raggruppato)

   MUNICIPIO  poi     length  Incidenti  tot      % poi
0          1   62  30.399490        859   62  48.818898
2          3    9   8.259516       1014    9   7.086614
8          9    9  17.617323        947    9   7.086614
3          4    9   8.233088        691    9   7.086614
4          5    9   8.277687        638    9   7.086614
1          2    9   8.960897        567    9   7.086614
7          8    8  11.738548        996    8   6.299213
5          6    7   8.413570        698    7   5.511811
6          7    5   4.381736        806    5   3.937008


Dettaglio sulla qualità del tipo di POI

In [17]:
gdf_analisi_cultura_spettacolo = gdf_analisi_cultura_spettacolo[gdf_analisi_cultura_spettacolo["tipo"] != "Strade_ciclabili"]
gdf_analisi_cultura_spettacolo_raggruppato = gdf_analisi_cultura_spettacolo.groupby("tipo").agg({
    "poi": "sum"}).reset_index().sort_values(["poi"], ascending=[False])
gdf_analisi_cultura_spettacolo_raggruppato

,tipo,poi
2,Teatri,76
1,Musei,28
0,Cinema,23


## Sanità

Utilizzeremo i seguenti dati: 

1. **Ospedali**
2. **famracie**


Saranno i nostri "poi", cioè "Point of Interest"
che verranno aggiunti al Grafo della rete 
Ciclabile/Stradale per la ricerca di percorsi

### Analisi Completa

In [29]:
gdf_list = [
    {"gdf":  gpd.read_file(PATH_OSPEDALI_CLEAN),
     "tipo": "ospedali",
     "attr": {"priorita": 1}},
     {"gdf":  gpd.read_file(PATH_FARMACIE_CLEAN),
     "tipo": "farmacie",
     "attr": {"priorita": 2}}
     ]
Graph_functions.auto_analysis_poi(gdf_list,
                                  PATH_PICKLE=PATH_SANITA_PICKLE_STAGING,
                                  PATH_GEOJSON=PATH_SANITA_CLEAN)

### Analisi Filtrata

In [32]:
with open(PATH_SANITA_PICKLE_STAGING, "rb") as file:
    G_sanita = pickle.load(file)

Graph_functions.auto_filter_poi(G_percorsi=G_sanita,
                                max_pois=300,
                                peso_inquinamento_vs_incidenti=0,
                                peso_fattori_esterni_vs_distribuzione_poi=1,
                                PATH_GEOJSON=PATH_SANITA_FILTRATO_CLEAN.replace("clean.geojson", "incidenti_clean.geojson"))

In [33]:
with open(PATH_SANITA_PICKLE_STAGING, "rb") as file:
    G_sanita = pickle.load(file)

Graph_functions.auto_filter_poi(G_percorsi=G_sanita,
                                max_pois=300,
                                peso_inquinamento_vs_incidenti=1,
                                peso_fattori_esterni_vs_distribuzione_poi=1,
                                PATH_GEOJSON=PATH_SANITA_FILTRATO_CLEAN.replace("clean.geojson", "inquinamento_clean.geojson"))

In [34]:
gdf_completo = gpd.read_file(PATH_SANITA_CLEAN)
gdf_filtrato_incidenti = gpd.read_file(PATH_SANITA_FILTRATO_CLEAN.replace("clean.geojson", "incidenti_clean.geojson"))
gdf_filtrato_inquinamento = gpd.read_file(PATH_SANITA_FILTRATO_CLEAN.replace("clean.geojson", "inquinamento_clean.geojson"))

gdf_completo["origine"] = "completo"
gdf_filtrato_incidenti["origine"] = "solo_incidenti"
gdf_filtrato_inquinamento["origine"] = "solo_inquinamento"
gdf_unito = gpd.GeoDataFrame(gpd.pd.concat([
        gdf_completo,
        gdf_filtrato_incidenti,
        gdf_filtrato_inquinamento,
    ], ignore_index=True), crs=CRS_GRAD)

In [37]:
gdf_unito.to_file(PATH_SANITA_FILTRATO_CLEAN.replace("clean.geojson", "unito_clean.geojson"))

### Statistiche

In [31]:
gdf_analisi_ospedali = gpd.read_file(PATH_SANITA_CLEAN, driver= "GeoJSON")

c:\Users\apera\OneDrive\Documenti\progetti\Progetto-Bike\.venv\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver GeoJSON does not support open option DRIVER
  return ogr_read(


In [32]:
# Eliminiamo connessioni "artificiali" usate per connettere i "poi" al grafo delle strade
gdf_analisi_ospedali_mod = gdf_analisi_ospedali[gdf_analisi_ospedali["artificial"] != True]

# Eliminiamo i percorsi completamente contenuti all'interno dei "poi" poligonari: parchi
gdf = gpd.read_file(PATH_OSPEDALI_CLEAN)
geometry_parchi = gdf.union_all()
gdf_senza_poi = []
for idx, row in gdf_analisi_ospedali_mod.iterrows():
    if not geometry_parchi.contains(row.geometry):
        gdf_senza_poi.append(row)
gdf_analisi_ospedali_mod = gpd.GeoDataFrame(gdf_senza_poi, crs=CRS_GRAD)

# Raggruppiamo per "highway" e calcoliamo (in km -> /1000) la lunghezza complessiva delle strade
gdf_analisi_ospedali_mod_most_highways = gdf_analisi_ospedali_mod.to_crs(CRS_METR).groupby("highway").agg({
    "length": lambda x: sum(x/1000)
    }).sort_values(by="length", ascending=False)

In [33]:
# Stampiamo il risultato
print(gdf_analisi_ospedali_mod_most_highways)
print(f"Tot: {gdf_analisi_ospedali_mod_most_highways["length"].sum()}")

                              length
highway                             
footway                   100.647292
cycleway                   78.490197
residential                26.678204
tertiary                   25.932165
secondary                  11.969441
primary                     9.613974
service                     3.456572
track                       2.130879
pedestrian                  1.134876
residential footway         0.653908
footway steps               0.511464
unclassified                0.373563
path                        0.355212
footway service             0.296168
cycleway footway            0.278997
tertiary_link               0.183503
residential secondary       0.146892
residential pedestrian      0.146128
residential cycleway        0.116950
footway platform            0.094960
footway pedestrian          0.087068
steps path                  0.084408
unclassified residential    0.040639
living_street footway       0.038044
secondary_link              0.034092
p

Numero poi per municipio che sono stati connessi e % poi connessi in ogni municipio rispetto al totale connesso

In [38]:
gdf_analisi_sanita_raggruppato = gdf_analisi_ospedali.groupby(["MUNICIPIO"]).agg({
    "poi": "sum",
    "length": lambda x: sum(x)/1000
}).reset_index()
gdf_analisi_sanita_raggruppato["MUNICIPIO"] = gdf_analisi_sanita_raggruppato["MUNICIPIO"].astype(int)
gdf_analisi_sanita_raggruppato = gdf_analisi_sanita_raggruppato.sort_index()
gdf_analisi_sanita_raggruppato = gdf_analisi_sanita_raggruppato.merge(
    gpd.read_file(PATH_INCIDENTI_CLEAN)[["MUNICIPIO", "Incidenti"]], on="MUNICIPIO")

gdf_pois = gpd.read_file(PATH_SCORE_X_MUNICIPI)[["MUNICIPIO", "ospedali", "farmacie"]]
gdf_pois["tot"] = gdf_pois.drop("MUNICIPIO", axis=1).sum(axis=1)
gdf_pois["MUNICIPIO"] = gdf_pois["MUNICIPIO"].astype(int)
gdf_analisi_sanita_raggruppato = gdf_analisi_sanita_raggruppato.merge(gdf_pois[["MUNICIPIO", "tot"]], on="MUNICIPIO")
somma_poi = gdf_analisi_sanita_raggruppato["poi"].sum()
gdf_analisi_sanita_raggruppato["% poi"] = gdf_analisi_sanita_raggruppato.apply(lambda row: row["poi"] / somma_poi * 100, axis=1)
gdf_analisi_sanita_raggruppato.sort_values(["poi", "Incidenti", "length"], ascending=False, inplace=True)
print(gdf_analisi_sanita_raggruppato)


gdf_analisi_sanita = gdf_analisi_ospedali[gdf_analisi_ospedali["tipo"] != "Strade_ciclabili"]
gdf_analisi_sanita_raggruppato = gdf_analisi_ospedali.groupby("tipo").agg({
    "poi": "sum"}).reset_index().sort_values(["poi"], ascending=[False])
gdf_analisi_sanita_raggruppato

   MUNICIPIO  poi     length  Incidenti  tot      % poi
0          1   76  33.241180        859   78  16.414687
2          3   57  25.468327       1014   57  12.311015
7          8   56  42.442970        996   56  12.095032
3          4   53  32.164410        691   52  11.447084
8          9   51  30.706032        947   51  11.015119
6          7   51  35.619282        806   51  11.015119
1          2   42  26.588773        567   42   9.071274
5          6   41  26.085688        698   41   8.855292
4          5   36  24.509263        638   35   7.775378


,tipo,poi
0,Farmacie,423
1,Ospedali,40
2,Strade_ciclabili,0


## Statistiche POI x Municipi

In [16]:
# Carica dati, aggiungendo una colonna "tipo" e tenendo solo questa colonna e geometry
municipi = gpd.read_file(PATH_MUNICIPI_CLEAN)
parchi = gpd.read_file(PATH_PARCHI_CLEAN).assign(tipo="parchi")[["tipo", "geometry"]]
fontane = gpd.read_file(PATH_FONTANE_CLEAN).assign(tipo="fontane")[["tipo", "geometry"]]
impianti = gpd.read_file(PATH_IMPIANTI_SPORTIVI_CLEAN).assign(tipo="impianti_sportivi")[["tipo", "geometry"]]
scuole = gpd.read_file(PATH_SCUOLE_CLEAN).assign(tipo="scuole")[["tipo", "geometry"]]
aree_gioco = gpd.read_file(PATH_AREE_GIOCO_CLEAN).assign(tipo="aree_gioco")[["tipo", "geometry"]]
biblioteche = gpd.read_file(PATH_BIBLIOTECHE_CLEAN).assign(tipo="biblioteche")[["tipo", "geometry"]]
stazioni_bikemi = gpd.read_file(PATH_BIKEMI_CLEAN).assign(tipo="stazioni_bikemi")[["tipo", "geometry"]]
case_acqua = gpd.read_file(PATH_CASE_ACQUA_CLEAN).assign(tipo="case_acqua")[["tipo", "geometry"]]
cinema = gpd.read_file(PATH_CINEMA_CLEAN).assign(tipo="cinema")[["tipo", "geometry"]]
farmacie = gpd.read_file(PATH_FARMACIE_CLEAN).assign(tipo="farmacie")[["tipo", "geometry"]]
musei = gpd.read_file(PATH_MUSEI_CLEAN).assign(tipo="musei")[["tipo", "geometry"]]
ospedali = gpd.read_file(PATH_OSPEDALI_CLEAN).assign(tipo="ospedali")[["tipo", "geometry"]]
teatri = gpd.read_file(PATH_TEATRI_CLEAN).assign(tipo="teatri")[["tipo", "geometry"]]
# Anche le ciclabili come "poi" per poter poi filtrare con le analisi
gdf_ciclabili = gpd.read_file(PATH_STRADE_CICLABILI_GEOJSON_CLEAN)
gdf_ciclabili_cycleway = gdf_ciclabili[gdf_ciclabili["highway"] == "cycleway"]
gdf_point_ciclabili = []
for idx, row in gdf_ciclabili_cycleway.iterrows():
    gdf_point_ciclabili.append(Point(row.geometry.coords[0]))
    gdf_point_ciclabili.append(Point(row.geometry.coords[-1]))
ciclabili = gpd.GeoDataFrame(gdf_point_ciclabili, columns=["geometry"], crs=CRS_GRAD).assign(tipo="ciclabile")[["tipo", "geometry"]]

Uniamo le righe dei vari dataframe in uno unico

In [17]:
pois = gpd.pd.concat([
    parchi, fontane, impianti, scuole, aree_gioco, biblioteche,
    stazioni_bikemi, case_acqua, cinema, farmacie, musei, ospedali, teatri, ciclabili
], ignore_index=True)

Tramite sjoin con predicato "intersect": eseguiamo una left-join con il gdf dei municipi in base
all'intersezione delle geometrie

In [18]:
pois_con_municipio = gpd.sjoin(
    municipi,
    pois,
    how="left",
    predicate="intersects"
    ).drop(["index_right"], axis=1) # eliminiamo l'index_right (conserva l'indice del dataframe del gdf di destra ma non ci serve)

Raggruppiamo per Municipio e Tipo (mettiamo anche geometry così ci rimane per il geojson).  

In [82]:
conteggi = pois_con_municipio.groupby(['MUNICIPIO', 'geometry', 'tipo']).size().unstack(fill_value=0).reset_index()

Aggiungiamo una colonna "point_etichetta" a cui assegniamo il "centroid" del poligono dei vari municipi, in modo  
da utilizzarlo su kepler per aggiungere un layer "poin" a cui assegniamo una "label" (etichetta) così visualizziamo  
a schermo il numero dell'attributo preso in considerazione (esempio, numero di parchi = 34, così si vede direttamente sulla mappa  
oltre a vedersi solo il colore. Può essere carino).

In [20]:
conteggi["point_etichetta"] = conteggi.geometry.apply(lambda x: x.centroid)

Riconvertiamo in geodataframe e salviamo su file geojson

In [21]:
conteggi = gpd.GeoDataFrame(conteggi, geometry="geometry", crs=CRS_GRAD)
conteggi.to_file(PATH_SCORE_X_MUNICIPI, driver="GeoJSON")